In [1]:
#Carga de datos
import pandas as pd
from sklearn import preprocessing
import numpy as np
data = pd.read_csv('DS_Credito Hipotecario.csv', encoding= "ISO-8859-1", sep=",")
data.head(20)

,ID,SEXO,FLAG_CASADO,NRO_DEPENDIENTES,EDUCACION,FLAG_TRAB_INDEP,INGRESOS_SOLICITANTE,INGRESOS_COSOLICITANTE,MONTO_PRESTAMO_MILES,PLAZO_PRESTAMO_MESES,FLAG_HISTORIAL_CREDITICIO,TIPO_ZONA,FLAG_CRED_HIPO
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,Bueno,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,Bueno,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,Bueno,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,Bueno,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,Bueno,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,Bueno,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,Bueno,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,Malo,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,Bueno,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,Bueno,Semiurban,N


In [2]:
class PreProcessing:
    def __init__(self, data):
        self.data = data.copy()
        columns_data = list(data.columns)
        self.columns = columns_data[1:-1]
        self.target = columns_data[-1]
        
    @property
    def columns_null(self):
        return [
            col for col in self.columns if self.data[col].isnull().any()
        ]   
    
    @property
    def colums_null_for_types(self):
        return {
            str(k).lower(): [col for col in list(v) if col in self.columns_null]
            for k, v in 
            data.columns.to_series().groupby(data.dtypes).groups.items()
        }

    @property
    def types(self):
        return [str(tipo) for tipo in set(data.dtypes)]
        
    def verificator_null(self):
        if self.data.isnull().any().any():
            print(pd.concat(
                [self.data.isnull().any(), 
                 self.data.isnull().sum(),
                 self.data.dtypes], 
                axis=1))
        else:
            print("DATA LIMPIA DE NULOS")
            
    def describe_data(self):
        print(data.describe(include="all"))
    
    def compare_with_target(self, action='count'):
        for colum in self.columns:
            print(data.pivot_table(
                index=[colum], 
                columns=[self.target],
                values=['ID'],
                aggfunc=action
            ))
            print("//"*50)
    
    def clean_data_v2(self):
        for type_data, list_col in self.colums_null_for_types.items():
            for col in list_col:
                if type_data == 'object':
                    self.data[col].fillna('NO PRECISA', inplace=True)
                else:
                    self.data[col].fillna(data[col].mean(), inplace=True)

    def clean_data_v1(self):
        for type_data, list_col in self.colums_null_for_types.items():
            for col in list_col:
                if type_data == 'object':
                    self.data[col].fillna(data[col].mode()[0], inplace=True)
                else:
                    self.data[col].fillna(data[col].mean(), inplace=True)
    
    def analysis_percentile(self, list_columns, percent):
        for col in list_columns:
            valor_in_percent = np.nanpercentile(data[col], [percent])[0]
            indice_percen = self.data[col] > valor_in_percent
            self.data.loc[indice_percen, col] = valor_in_percent
            print(col, ' recortada, nuevo masximo :', max(self.data[col]))
            
    def factorize_data(self, list_columns, cut=False, range_data=range(0, 20000, 1000)):
        for col in list_columns:
            data_column = pd.cut(self.data[col], range_data) if cut else self.data[col]                
            data_factorize = pd.factorize(data_column)[0]    
            range_train = preprocessing.LabelEncoder().fit(data_factorize)
            self.data[col] = range_train.transform(data_factorize)
            print(col, 'factorizada')
            
    def save_data(self, sufijo=''):
        self.data = self.data.drop(['ID'], axis=1)
        self.data.to_csv("CLEAN_credito{0}.csv".format(sufijo), index=False)
    

In [3]:
#Creo dos instancias de mi data segun el metodo de preprocesado seleccionado
process_1 = PreProcessing(data)
process_2 = PreProcessing(data)

In [4]:
#Analizo los nulos
process_1.verificator_null()

                               0   1        2
ID                         False   0   object
SEXO                        True  13   object
FLAG_CASADO                 True   3   object
NRO_DEPENDIENTES            True  15   object
EDUCACION                  False   0   object
FLAG_TRAB_INDEP             True  32   object
INGRESOS_SOLICITANTE       False   0    int64
INGRESOS_COSOLICITANTE      True  61  float64
MONTO_PRESTAMO_MILES        True  22  float64
PLAZO_PRESTAMO_MESES        True  14  float64
FLAG_HISTORIAL_CREDITICIO   True  50   object
TIPO_ZONA                  False   0   object
FLAG_CRED_HIPO             False   0   object


In [5]:
#Obtento una descripcion de las columnas a detalle
process_1.describe_data()

              ID  SEXO FLAG_CASADO NRO_DEPENDIENTES EDUCACION FLAG_TRAB_INDEP  \
count        614   601         611              599       614             582   
unique       614     2           2                4         2               2   
top     LP002178  Male         Yes                0  Graduate              No   
freq           1   489         398              345       480             500   
mean         NaN   NaN         NaN              NaN       NaN             NaN   
std          NaN   NaN         NaN              NaN       NaN             NaN   
min          NaN   NaN         NaN              NaN       NaN             NaN   
25%          NaN   NaN         NaN              NaN       NaN             NaN   
50%          NaN   NaN         NaN              NaN       NaN             NaN   
75%          NaN   NaN         NaN              NaN       NaN             NaN   
max          NaN   NaN         NaN              NaN       NaN             NaN   

        INGRESOS_SOLICITANT

In [6]:
#Analizamos un analisis opcional de los datos vs la variable TARGET
process_1.compare_with_target()

                 ID     
FLAG_CRED_HIPO    N    Y
SEXO                    
Female           37   75
Male            150  339
////////////////////////////////////////////////////////////////////////////////////////////////////
                 ID     
FLAG_CRED_HIPO    N    Y
FLAG_CASADO             
No               79  134
Yes             113  285
////////////////////////////////////////////////////////////////////////////////////////////////////
                   ID     
FLAG_CRED_HIPO      N    Y
NRO_DEPENDIENTES          
0                 107  238
1                  36   66
2                  25   76
3+                 18   33
////////////////////////////////////////////////////////////////////////////////////////////////////
                 ID     
FLAG_CRED_HIPO    N    Y
EDUCACION               
Graduate        140  340
Not Graduate     52   82
////////////////////////////////////////////////////////////////////////////////////////////////////
                  ID     
FLAG_C

In [7]:
## INiciamos el proceso de LIMPIADO DE NULOS
process_1.clean_data_v1()
process_2.clean_data_v2()

In [8]:
process_1.verificator_null()

DATA LIMPIA DE NULOS


In [9]:
process_2.verificator_null()

DATA LIMPIA DE NULOS


In [10]:
process_1.analysis_percentile(['INGRESOS_COSOLICITANTE', 'MONTO_PRESTAMO_MILES', 'INGRESOS_SOLICITANTE'], 95)
print("=="*5)
process_2.analysis_percentile(['INGRESOS_COSOLICITANTE', 'MONTO_PRESTAMO_MILES', 'INGRESOS_SOLICITANTE'], 95)

INGRESOS_COSOLICITANTE  recortada, nuevo masximo : 5144.799999999996
MONTO_PRESTAMO_MILES  recortada, nuevo masximo : 297.7999999999997
INGRESOS_SOLICITANTE  recortada, nuevo masximo : 14583.0
INGRESOS_COSOLICITANTE  recortada, nuevo masximo : 5144.799999999996
MONTO_PRESTAMO_MILES  recortada, nuevo masximo : 297.7999999999997
INGRESOS_SOLICITANTE  recortada, nuevo masximo : 14583.0


In [11]:
process_1.factorize_data(['INGRESOS_COSOLICITANTE', 'INGRESOS_SOLICITANTE'], cut=True)
print("=="*5)
process_2.factorize_data(['INGRESOS_COSOLICITANTE', 'INGRESOS_SOLICITANTE'], cut=True)

INGRESOS_COSOLICITANTE factorizada
INGRESOS_SOLICITANTE factorizada
INGRESOS_COSOLICITANTE factorizada
INGRESOS_SOLICITANTE factorizada


In [12]:
process_1.factorize_data(['MONTO_PRESTAMO_MILES'], cut=True, range_data=range(0, 500, 25))
print("=="*5)
process_2.factorize_data(['MONTO_PRESTAMO_MILES'], cut=True, range_data=range(0, 500, 25))

MONTO_PRESTAMO_MILES factorizada
MONTO_PRESTAMO_MILES factorizada


In [14]:
process_1.factorize_data(
    ['SEXO', 'FLAG_CASADO', 'NRO_DEPENDIENTES', 'EDUCACION', 'FLAG_CRED_HIPO',
     'FLAG_TRAB_INDEP', 'PLAZO_PRESTAMO_MESES', 'FLAG_HISTORIAL_CREDITICIO', 'TIPO_ZONA'])
print("=="*5)
process_2.factorize_data(
    ['SEXO', 'FLAG_CASADO', 'NRO_DEPENDIENTES', 'EDUCACION', 'FLAG_CRED_HIPO',
     'FLAG_TRAB_INDEP', 'PLAZO_PRESTAMO_MESES', 'FLAG_HISTORIAL_CREDITICIO', 'TIPO_ZONA'])

SEXO factorizada
FLAG_CASADO factorizada
NRO_DEPENDIENTES factorizada
EDUCACION factorizada
FLAG_CRED_HIPO factorizada
FLAG_TRAB_INDEP factorizada
PLAZO_PRESTAMO_MESES factorizada
FLAG_HISTORIAL_CREDITICIO factorizada
TIPO_ZONA factorizada
SEXO factorizada
FLAG_CASADO factorizada
NRO_DEPENDIENTES factorizada
EDUCACION factorizada
FLAG_CRED_HIPO factorizada
FLAG_TRAB_INDEP factorizada
PLAZO_PRESTAMO_MESES factorizada
FLAG_HISTORIAL_CREDITICIO factorizada
TIPO_ZONA factorizada


In [15]:
process_1.save_data(sufijo='_1')
process_1.data.head()

,SEXO,FLAG_CASADO,NRO_DEPENDIENTES,EDUCACION,FLAG_TRAB_INDEP,INGRESOS_SOLICITANTE,INGRESOS_COSOLICITANTE,MONTO_PRESTAMO_MILES,PLAZO_PRESTAMO_MESES,FLAG_HISTORIAL_CREDITICIO,TIPO_ZONA,FLAG_CRED_HIPO
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,1,1,0,0,0,1,1
2,0,1,0,0,1,2,0,1,0,0,0,0
3,0,1,0,1,0,2,2,2,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
process_2.save_data(sufijo='_2')
process_2.data.head()

,SEXO,FLAG_CASADO,NRO_DEPENDIENTES,EDUCACION,FLAG_TRAB_INDEP,INGRESOS_SOLICITANTE,INGRESOS_COSOLICITANTE,MONTO_PRESTAMO_MILES,PLAZO_PRESTAMO_MESES,FLAG_HISTORIAL_CREDITICIO,TIPO_ZONA,FLAG_CRED_HIPO
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,1,1,0,0,0,1,1
2,0,1,0,0,1,2,0,1,0,0,0,0
3,0,1,0,1,0,2,2,2,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
